In [ ]:
import numpy as np
import numba
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from plotting import plot

In [ ]:
thrust = 210000.0
g0 = 1.61544
STANDARD_GRAV = 9.81
Isp = 265.2 
vch_params = (thrust, g0, Isp)

x0 = np.array([0,0,0,0,50000])

In [ ]:
@numba.njit
def dynamics(x, u, params):

    thrust = params[0]
    g0 = params[1]
    Isp = params[2]
    u_ = np.append(u, u[-1])
    cos_theta  = np.cos(u_)
    sin_theta  = np.sin(u_)
    dx = np.zeros_like(x)
    dx[:,0] = x[:,2]
    dx[:,1] = x[:,3]
    dx[:,2] =  (thrust * cos_theta) / x[:,4] 
    dx[:,3] =  (thrust * sin_theta)  / x[:,4] - g0
    dx[:,4] =  -thrust / STANDARD_GRAV / Isp
    return dx


# Define the dynamics defects
@numba.njit
def state_defects(decision_variables, args):
    N = args[0] 
    states_dim = args[1] 
    params = args[2]
    # Index [0 N )in the decision_variables vector contains the control inputs 
    u = decision_variables[:N]
    # Index [N -1] in the decision_variables vector contains the state  
    x = decision_variables[N:-1].reshape((N+1, states_dim))
    dt = decision_variables[-1] # Time is the last decision var here
    # Calculate the dynamics
    x_dot = dynamics(x, u, params)
    # Calculate the approximation of integral using trapezoidal quadrature
    integral = ((x_dot[:-1] + x_dot[1:])) / 2 * dt
    # Calculate the state defects
    defects = x[1:] - x[:-1]  - integral
    return defects.transpose().flatten()

In [ ]:
# Define the initial and final states 
x0 = np.array([0,0,0,0,50000])
terminal_x = [None, 1.8500e+05, 1.6270e+03, 0, None]
_, xf_y, xf_vx, xf_vy, _ = terminal_x

states_dim = 5
N = 25 # Number of Scollocation points, 

#bounds of pitch angle
u_b = np.pi/2
l_b = -np.pi/2

In [ ]:
np.pi/2

In [ ]:
mdot = thrust / STANDARD_GRAV / Isp
t_max = 50000 / mdot
dt_max = t_max/ N

In [ ]:
# Initial guess for control inputs
t_init  = np.linspace(0, 402, N+1) 

u_init = np.linspace(1.3, -0.76, N) # Decision variable
# u_init = np.arctan(-0.01667*t_init[:-1]+ 4.89202)

dt_init = t_init[1] - t_init[0]  # time step

# Initial guess for states
x_init = np.zeros((N+1, states_dim))
x_init[:, 0] = np.linspace(x0[0], 215e3, N+1) 
x_init[:, 1] = np.linspace(0, xf_y, N+1)
x_init[:, 2] = np.linspace(0, xf_vx, N+1)
x_init[:, 3] = np.linspace(0, xf_vy, N+1)
x_init[:, 4] = np.linspace(50000, 0.2 *  50000, N+1)

# Concatenate control inputs and states into a single decision variable
initial_guess = np.concatenate([u_init, x_init.flatten(), [dt_init]])


In [ ]:
len(initial_guess) 

In [ ]:
@numba.njit
def objective(decision_variables, args):
    N = args[0] 
    u = decision_variables[:N]
    # Index [N -1] in the decision_variables vector contains the state  
    x = decision_variables[N:-1].reshape((N+1, states_dim))
    dt = decision_variables[-1] # Time is the last decision var here
    defects = state_defects(decision_variables, args)
    return -np.sum((x[:,4]/500)**2)  + 10*dt *N + np.sum(defects )**2

# Define the optimization problem
def problem(decision_variables, args):
    N = args[0] 
    u = decision_variables[:N]
    # Index [N -1] in the decision_variables vector contains the state  
 
    obj_value = objective(decision_variables, args)
    
    return obj_value

# Define the bounds for the decision variables
bounds = [(l_b, u_b)] * N # this case it is the control parameter for the trajectory, the pitch command
num_state_bounds = states_dim*(N+1)

state_bounds = [(0, np.inf)] * num_state_bounds

# for i in range(0,N+1):
#     state_bounds[states_dim*i] = (0,None)

xf_y, xf_vx, xf_vy
bounds = bounds + state_bounds    
#Enforcing Bound constraint on initial and final states
bounds[N]    = (0.0,0.0) # x 
bounds[N+1]  = (0.0,0.0) # y
bounds[N+2]  = (0.0, 0.0) # v_x 
bounds[N+3] = (0.0, 0.0) # v_y 
bounds[N+4] = (50000, 50000) # mass

# Final Bounds
bounds[N+ num_state_bounds - states_dim + 0] = (0.0,np.inf) # x 
bounds[N+ num_state_bounds - states_dim + 1] = (xf_y,xf_y) # y
bounds[N+ num_state_bounds - states_dim + 2] = (xf_vx, xf_vx) # v_x 
bounds[N+ num_state_bounds - states_dim + 3] = (xf_vy, xf_vy) # v_y 
bounds[N+ num_state_bounds - states_dim + 4] = (0, 30000) # mass
bounds = bounds + [(dt_max/2, dt_max)] # Adding time bound 

arguments = ((N,states_dim, vch_params),)


# Define the constraints
constraints = [{'type': 'eq', 'fun': state_defects, 'args':arguments },]


In [ ]:
result = minimize(problem, initial_guess, method='SLSQP', bounds=bounds, args=arguments, constraints=constraints, options = {"maxiter": 1000, "disp": True})
print(result)


In [ ]:
u = result.x[:N]
# Index [N -1] in the decision_variables vector contains the state  
x = result.x[N:-1].reshape((N+1, states_dim))
dt = result.x[-1] # Time is the last decision var here

In [ ]:
t = np.linspace(0, N*dt, N+1) 

In [ ]:
len(t)

In [ ]:
plot(
    t, [u],
    title="Time vs Pitch steering", 
    xlabel="Time", 
    ylabel="theta",
    )

In [ ]:
plot(
    t,[x[:,0], x[:,1]], y2 = [x[:,2], x[:,3]],
    title="Time vs States", 
    xlabel="Time", 
    ylabel=("Pos", "Vel"),
    trace_names=("pos_x", "pos_y", "vel_x", "vel_y")
    )

In [ ]:
plot( t, [x[:,4]], title="time vs Mass", xlabel="Time", ylabel="Mass")